# 📊 Pipelines V4 Production - Architecture

Ce notebook définit **8 pipelines individuels** et **4 pipelines nested** pour le projet COVID-19.

---

## 🏗️ Architecture Hiérarchique des Transformateurs

```
📦 TRANSFORMATEURS COMMUNS (utilisés par tous)
│
├─ 🔍 ImagePathLoader          → Scan directories et récupère chemins
├─ 📋 TupleToDataFrame          → Convertit tuples en DataFrame
└─ 📊 ImageAnalyser             → Load images + analyse métadonnées

📦 PREPROCESSING (selon type de modèle)
│
├─ 🖼️  ML Classique (64x64, grayscale)
│   ├─ ImageResizer (64x64)
│   ├─ ImageNormalizer [0,1]
│   └─ RGB_to_L (grayscale)
│
├─ 🧠 CNN Custom (128x128, RGB)
│   ├─ ImageResizer (128x128)
│   └─ ImageNormalizer [0,1]
│
└─ 🔄 Transfer Learning (224x224, RGB)
    ├─ ImageResizer (224x224)
    └─ ImageNormalizer [0,1]

📦 FEATURES EXTRACTION (pour ML uniquement)
│
├─ 📐 PCA Features
│   ├─ ImageFlattener
│   ├─ ImagePCA (50 components)
│   └─ PCAVisualizer (NEW)
│
└─ 📊 Histogram Features
    ├─ ImageHistogram (32 bins)
    └─ HistogramVisualizer (NEW)

📦 AUGMENTATION (optionnel)
│
└─ 🎨 Data Augmentation
    ├─ ImageAugmenter (rotation, brightness, noise, zoom)
    ├─ ImageMasker (optionnel)
    └─ ImageComparisonVisualizer (NEW)

📦 UTILITIES
│
├─ 🎯 TrainTestSplitter         → Split stratifié
├─ 📈 DatasetStatistics (NEW)   → Stats + visualisations
└─ 💾 SaveTransformer           → Sauvegarde intermédiaire
```

---

## 🎯 Matrice d'Utilisation des Pipelines

| Transformateur | Pipeline 1<br/>Exploration | Pipeline 2<br/>Preprocessing ML | Pipeline 3<br/>PCA | Pipeline 4<br/>Histogram | Pipeline 5<br/>Augmentation | Pipeline 6<br/>CNN | Pipeline 7<br/>Transfer | Pipeline 8<br/>Split |
|----------------|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| **ImagePathLoader** | ✅ | ❌ | ❌ | ❌ | ❌ | ❌ | ❌ | ❌ |
| **TupleToDataFrame** | ✅ | ❌ | ❌ | ❌ | ❌ | ❌ | ❌ | ❌ |
| **ImageAnalyser** | ✅ | ✅ | ❌ | ❌ | ❌ | ✅ | ✅ | ❌ |
| **DatasetStatistics** | ✅ | ❌ | ❌ | ❌ | ❌ | ❌ | ❌ | ❌ |
| **ImageResizer** | ❌ | ✅<br/>64x64 | ❌ | ❌ | ❌ | ✅<br/>128x128 | ✅<br/>224x224 | ❌ |
| **ImageNormalizer** | ❌ | ✅ | ❌ | ❌ | ❌ | ✅ | ✅ | ❌ |
| **RGB_to_L** | ❌ | ✅ | ❌ | ❌ | ❌ | ❌ | ❌ | ❌ |
| **ImageFlattener** | ❌ | ❌ | ✅ | ❌ | ❌ | ❌ | ❌ | ❌ |
| **ImagePCA** | ❌ | ❌ | ✅ | ❌ | ❌ | ❌ | ❌ | ❌ |
| **PCAVisualizer** | ❌ | ❌ | ✅ | ❌ | ❌ | ❌ | ❌ | ❌ |
| **ImageHistogram** | ❌ | ❌ | ❌ | ✅ | ❌ | ❌ | ❌ | ❌ |
| **HistogramVisualizer** | ❌ | ❌ | ❌ | ✅ | ❌ | ❌ | ❌ | ❌ |
| **ImageAugmenter** | ❌ | ❌ | ❌ | ❌ | ✅ | ❌ | ❌ | ❌ |
| **ImageComparisonViz** | ❌ | ✅ | ❌ | ❌ | ✅ | ❌ | ❌ | ❌ |
| **TrainTestSplitter** | ❌ | ❌ | ❌ | ❌ | ❌ | ❌ | ❌ | ✅ |

---

## 🔗 Nested Pipelines (Combinaisons)

```
📦 NESTED 1: ML Complete (PCA)
Pipeline 1 (Exploration) → Pipeline 2 (Preprocessing ML) → Pipeline 3 (Features PCA)
└─ Output: Features PCA (n_samples, 50) prêtes pour RandomForest/XGBoost

📦 NESTED 2: ML Complete (Histogrammes)
Pipeline 1 (Exploration) → Pipeline 2 (Preprocessing ML) → Pipeline 4 (Features Histogram)
└─ Output: Features Histogrammes (n_samples, 32) prêtes pour ML

📦 NESTED 3: CNN Complete
Pipeline 1 (Exploration) → Pipeline 6 (Preprocessing CNN)
└─ Output: Images 128x128x3 RGB prêtes pour CNN custom

📦 NESTED 4: Transfer Learning Complete
Pipeline 1 (Exploration) → Pipeline 7 (Preprocessing Transfer)
└─ Output: Images 224x224x3 RGB prêtes pour VGG16/ResNet/EfficientNet
```

---

## 📋 Workflows Typiques

### 🎯 Workflow ML Classique (RandomForest/XGBoost)
```
1. Pipeline 1 (Exploration) → Analyser dataset
2. Pipeline 8 (Split) → Séparer train/test
3. Pipeline 2 (Preprocessing ML) → Préparer images 64x64 gray
4. Pipeline 3 ou 4 → Extraire features PCA ou Histogrammes
5. Entraîner RandomForest/XGBoost
```

### 🧠 Workflow CNN Custom
```
1. Pipeline 1 (Exploration) → Analyser dataset
2. Pipeline 8 (Split) → Séparer train/test
3. Pipeline 5 (Augmentation) → Augmenter train set (optionnel)
4. Pipeline 6 (Preprocessing CNN) → Préparer images 128x128 RGB
5. Entraîner CNN à 5 blocs
```

### 🔄 Workflow Transfer Learning
```
1. Pipeline 1 (Exploration) → Analyser dataset
2. Pipeline 8 (Split) → Séparer train/test
3. Pipeline 5 (Augmentation) → Augmenter train set (optionnel)
4. Pipeline 7 (Preprocessing Transfer) → Préparer images 224x224 RGB
5. Appliquer preprocess_input() spécifique au modèle
6. Fine-tuner VGG16/ResNet/Inception/EfficientNet
```

---

## 1. Imports et Configuration

In [1]:
%matplotlib inline

import sys
import os
import joblib
from datetime import datetime

# Path setup
sys.path.insert(0, os.path.abspath('..'))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Import transformateurs existants V3
from src.features.St_Pipeline.Transformateurs import (
    # Loaders
    ImagePathLoader,
    TupleToDataFrame,
    
    # Preprocessing
    ImageResizer,
    ImageAugmenter,
    ImageNormalizer,
    ImageMasker,
    ImageFlattener,
    ImageStandardScaler,
    RGB_to_L,
    
    # Analyse et features
    ImageAnalyser,
    ImagePCA,
    ImageHistogram,
    
    # Utilities
    TrainTestSplitter,
)

# Import nouveaux transformateurs (à créer)
# Ces imports vont échouer jusqu'à ce qu'on crée les transformateurs
try:
    from src.features.St_Pipeline.Transformateurs import (
        DatasetStatistics,
        ImageComparisonVisualizer,
        PCAVisualizer,
        HistogramVisualizer,
    )
    NEW_TRANSFORMERS_AVAILABLE = True
    print("✅ Nouveaux transformateurs disponibles!")
except ImportError as e:
    print(f"⚠️ Nouveaux transformateurs pas encore implémentés: {e}")
    NEW_TRANSFORMERS_AVAILABLE = False

print("✅ Imports réussis!")
print(f"📁 Répertoire de travail: {os.getcwd()}")
print(f"🆕 Nouveaux transformateurs: {'✅' if NEW_TRANSFORMERS_AVAILABLE else '❌'}")

✅ Nouveaux transformateurs disponibles!
✅ Imports réussis!
📁 Répertoire de travail: /home/lena/DS_Covid/DS_COVID/notebooks
🆕 Nouveaux transformateurs: ✅


## 2. Configuration Globale

In [2]:
# =============================================================================
# CONFIGURATION GLOBALE
# =============================================================================

# Chemins
ROOT_DIR = "/home/lena/DS_Covid/DS_COVID/data/raw/COVID-19_Radiography_Dataset/COVID-19_Radiography_Dataset"
MODELS_DIR = "../models"
OUTPUTS_DIR = "../outputs"

# Créer les dossiers
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(OUTPUTS_DIR, exist_ok=True)

# Paramètres généraux
RANDOM_STATE = 42
TEST_SIZE = 0.2
VALIDATION_SIZE = 0.15  # Pour deep learning

# Paramètres images
IMG_SIZE_ML = (64, 64)          # Pour ML classique
IMG_SIZE_CNN = (128, 128)        # Pour CNN custom
IMG_SIZE_TRANSFER = (224, 224)   # Pour transfer learning

# Paramètres features
PCA_COMPONENTS = 50
HISTOGRAM_BINS = 32

# Classes du dataset
CLASSES = ['COVID', 'Lung_Opacity', 'Normal', 'Viral Pneumonia']

# Vérification dataset
if os.path.exists(ROOT_DIR):
    print(f"✅ Dataset trouvé: {ROOT_DIR}")
    labels = [d for d in os.listdir(ROOT_DIR) 
              if os.path.isdir(os.path.join(ROOT_DIR, d, 'images'))]
    print(f"📊 Labels disponibles: {labels}")
    print(f"📊 Nombre de classes: {len(labels)}")
    
    # Compter images
    total_images = 0
    for label in labels:
        img_dir = os.path.join(ROOT_DIR, label, 'images')
        if os.path.exists(img_dir):
            n_images = len([f for f in os.listdir(img_dir) if f.endswith('.png')])
            total_images += n_images
            print(f"   • {label}: {n_images} images")
    print(f"📊 Total: {total_images} images")
else:
    print(f"❌ Dataset introuvable: {ROOT_DIR}")
    raise FileNotFoundError("Dataset COVID-19 manquant")

print("\n" + "="*80)
print("✅ Configuration chargée avec succès!")
print("="*80)

✅ Dataset trouvé: /home/lena/DS_Covid/DS_COVID/data/raw/COVID-19_Radiography_Dataset/COVID-19_Radiography_Dataset
📊 Labels disponibles: ['Lung_Opacity', 'Normal', 'COVID', 'Viral Pneumonia']
📊 Nombre de classes: 4
   • Lung_Opacity: 6012 images
   • Normal: 10192 images
   • COVID: 3616 images
   • Viral Pneumonia: 1345 images
📊 Total: 21165 images

✅ Configuration chargée avec succès!


## 3. 📊 PIPELINE 1: Exploration Dataset

**Objectif**: Analyser le dataset et générer statistiques/visualisations

**Composants**:
1. ImagePathLoader - Scan directories
2. TupleToDataFrame - Convert to DataFrame
3. ImageAnalyser - Analyze metadata (NO image loading)
4. DatasetStatistics - Compute stats + visualizations (NOUVEAU)

In [3]:
print("\n" + "="*80)
print("🔍 PIPELINE 1: EXPLORATION DATASET")
print("="*80)

if NEW_TRANSFORMERS_AVAILABLE:
    pipeline_exploration = Pipeline([
        ('loader', ImagePathLoader(root_dir=ROOT_DIR, verbose=True, use_streamlit=False)),
        ('tuple_to_df', TupleToDataFrame(verbose=True, use_streamlit=False)),
        ('analyzer', ImageAnalyser(load_images=False, analyze_masks=True, verbose=True, use_streamlit=False)),
        ('statistics', DatasetStatistics(compute_pixel_stats=False, verbose=True, use_streamlit=False)),
    ])
else:
    # Version sans DatasetStatistics (pour l'instant)
    pipeline_exploration = Pipeline([
        ('loader', ImagePathLoader(root_dir=ROOT_DIR, verbose=True, use_streamlit=False)),
        ('tuple_to_df', TupleToDataFrame(verbose=True, use_streamlit=False)),
        ('analyzer', ImageAnalyser(load_images=False, analyze_masks=True, verbose=True, use_streamlit=False)),
    ])
    print("⚠️ DatasetStatistics pas encore implémenté - pipeline partiel")

# Sauvegarder (section, pas utilisable seule)
pipeline_path = os.path.join(MODELS_DIR, 'Section_1_exploration.pkl')
joblib.dump(pipeline_exploration, pipeline_path)
print(f"\n✅ Section sauvegardée: {pipeline_path}")
print("⚠️  Section non utilisable seule - utiliser Pipeline_ML_* ou Pipeline_CNN_*")

print("\n📋 Structure:")
for idx, (name, transformer) in enumerate(pipeline_exploration.steps, 1):
    print(f"  {idx}. {name}: {transformer.__class__.__name__}")

print("\n💡 Usage:")
print("  df_metadata = pipeline_exploration.fit_transform(None)")
print("  # Génère: DataFrame + visualisations statistiques")


🔍 PIPELINE 1: EXPLORATION DATASET

✅ Section sauvegardée: ../models/Section_1_exploration.pkl
⚠️  Section non utilisable seule - utiliser Pipeline_ML_* ou Pipeline_CNN_*

📋 Structure:
  1. loader: ImagePathLoader
  2. tuple_to_df: TupleToDataFrame
  3. analyzer: ImageAnalyser
  4. statistics: DatasetStatistics

💡 Usage:
  df_metadata = pipeline_exploration.fit_transform(None)
  # Génère: DataFrame + visualisations statistiques


## 4. 🔄 PIPELINE 2: Preprocessing pour ML Classique

**Objectif**: Préparer images pour RandomForest/XGBoost (64x64, grayscale)

**Composants**:
1. ImageAnalyser - Load images in memory
2. ImageResizer - Resize to 64x64
3. ImageNormalizer - Normalize to [0,1]
4. RGB_to_L - Convert to grayscale
5. ImageComparisonVisualizer - Show before/after (NOUVEAU - optionnel)

In [4]:
print("\n" + "="*80)
print("🖼️ PIPELINE 2: PREPROCESSING ML CLASSIQUE")
print("="*80)

transformers_list = [
    ('loader', ImageAnalyser(load_images=True, analyze_masks=False, verbose=True, use_streamlit=False)),
    ('resizer', ImageResizer(img_size=IMG_SIZE_ML, verbose=True, use_streamlit=False)),
    ('normalizer', ImageNormalizer(verbose=True, use_streamlit=False)),
    ('gray', RGB_to_L(verbose=True, use_streamlit=False)),
]

# Ajouter visualisateur si disponible
if NEW_TRANSFORMERS_AVAILABLE:
    transformers_list.append(
        ('visualizer', ImageComparisonVisualizer(n_samples=6, comparison_mode='side-by-side', 
                                                  verbose=True, use_streamlit=False))
    )

pipeline_preprocessing_ml = Pipeline(transformers_list)

# Sauvegarder (section, pas utilisable seule)
pipeline_path = os.path.join(MODELS_DIR, 'Section_2_preprocessing_ml.pkl')
joblib.dump(pipeline_preprocessing_ml, pipeline_path)
print(f"\n✅ Section sauvegardée: {pipeline_path}")
print("⚠️  Section non utilisable seule - utiliser Pipeline_ML_PCA ou Pipeline_ML_Histogram")

print("\n📋 Structure:")
for idx, (name, transformer) in enumerate(pipeline_preprocessing_ml.steps, 1):
    print(f"  {idx}. {name}: {transformer.__class__.__name__}")

print("\n💡 Usage:")
print("  df_processed = pipeline_preprocessing_ml.fit_transform(df_train)")
print("  # Images: 64x64, grayscale, normalisées")


🖼️ PIPELINE 2: PREPROCESSING ML CLASSIQUE

✅ Section sauvegardée: ../models/Section_2_preprocessing_ml.pkl
⚠️  Section non utilisable seule - utiliser Pipeline_ML_PCA ou Pipeline_ML_Histogram

📋 Structure:
  1. loader: ImageAnalyser
  2. resizer: ImageResizer
  3. normalizer: ImageNormalizer
  4. gray: RGB_to_L
  5. visualizer: ImageComparisonVisualizer

💡 Usage:
  df_processed = pipeline_preprocessing_ml.fit_transform(df_train)
  # Images: 64x64, grayscale, normalisées


## 5. 🔬 PIPELINE 3: Features PCA avec Visualisations

**Objectif**: Extraire features PCA + visualiser

**Composants**:
1. ImageFlattener - Flatten images to 1D
2. ImagePCA - Apply PCA (50 components)
3. PCAVisualizer - Visualize variance + projections 2D/3D (NOUVEAU)

In [5]:
print("\n" + "="*80)
print("📐 PIPELINE 3: FEATURES PCA")
print("="*80)

transformers_list = [
    ('flattener', ImageFlattener(verbose=True, use_streamlit=False)),
    ('pca', ImagePCA(n_components=PCA_COMPONENTS, verbose=True, use_streamlit=False)),
]

# Ajouter visualisateur si disponible
if NEW_TRANSFORMERS_AVAILABLE:
    transformers_list.append(
        ('pca_viz', PCAVisualizer(n_components=PCA_COMPONENTS, projection_mode='both', 
                                  verbose=True, use_streamlit=False))
    )

pipeline_features_pca = Pipeline(transformers_list)

# Sauvegarder (section, pas utilisable seule)
pipeline_path = os.path.join(MODELS_DIR, 'Section_3_features_pca.pkl')
joblib.dump(pipeline_features_pca, pipeline_path)
print(f"\n✅ Section sauvegardée: {pipeline_path}")
print("⚠️  Section non utilisable seule - utiliser Pipeline_ML_PCA")

print("\n📋 Structure:")
for idx, (name, transformer) in enumerate(pipeline_features_pca.steps, 1):
    print(f"  {idx}. {name}: {transformer.__class__.__name__}")

print("\n💡 Usage:")
print("  X_pca = pipeline_features_pca.fit_transform(df_processed, y=labels)")
print("  # Shape: (n_samples, 50)")
print("  # Génère: variance plot, 2D/3D projections")


📐 PIPELINE 3: FEATURES PCA

✅ Section sauvegardée: ../models/Section_3_features_pca.pkl
⚠️  Section non utilisable seule - utiliser Pipeline_ML_PCA

📋 Structure:
  1. flattener: ImageFlattener
  2. pca: ImagePCA
  3. pca_viz: PCAVisualizer

💡 Usage:
  X_pca = pipeline_features_pca.fit_transform(df_processed, y=labels)
  # Shape: (n_samples, 50)
  # Génère: variance plot, 2D/3D projections


## 6. 📊 PIPELINE 4: Features Histogrammes avec Visualisations

**Objectif**: Extraire histogrammes + visualiser

**Composants**:
1. ImageHistogram - Compute intensity histograms
2. HistogramVisualizer - Visualize distributions per class (NOUVEAU)

In [6]:
print("\n" + "="*80)
print("📊 PIPELINE 4: FEATURES HISTOGRAMMES")
print("="*80)

transformers_list = [
    ('histogram', ImageHistogram(bins=HISTOGRAM_BINS, verbose=True, use_streamlit=False)),
]

# Ajouter visualisateur si disponible
if NEW_TRANSFORMERS_AVAILABLE:
    transformers_list.append(
        ('hist_viz', HistogramVisualizer(n_bins=HISTOGRAM_BINS, plot_mode='all',
                                         verbose=True, use_streamlit=False))
    )

pipeline_features_histogram = Pipeline(transformers_list)

# Sauvegarder (section, pas utilisable seule)
pipeline_path = os.path.join(MODELS_DIR, 'Section_4_features_histogram.pkl')
joblib.dump(pipeline_features_histogram, pipeline_path)
print(f"\n✅ Section sauvegardée: {pipeline_path}")
print("⚠️  Section non utilisable seule - utiliser Pipeline_ML_Histogram")

print("\n📋 Structure:")
for idx, (name, transformer) in enumerate(pipeline_features_histogram.steps, 1):
    print(f"  {idx}. {name}: {transformer.__class__.__name__}")

print("\n💡 Usage:")
print("  X_hist = pipeline_features_histogram.fit_transform(df_processed, y=labels)")
print("  # Shape: (n_samples, 32)")
print("  # Génère: histogrammes moyens par classe, heatmaps")


📊 PIPELINE 4: FEATURES HISTOGRAMMES

✅ Section sauvegardée: ../models/Section_4_features_histogram.pkl
⚠️  Section non utilisable seule - utiliser Pipeline_ML_Histogram

📋 Structure:
  1. histogram: ImageHistogram
  2. hist_viz: HistogramVisualizer

💡 Usage:
  X_hist = pipeline_features_histogram.fit_transform(df_processed, y=labels)
  # Shape: (n_samples, 32)
  # Génère: histogrammes moyens par classe, heatmaps


## 7. 🎨 PIPELINE 5: Data Augmentation

**Objectif**: Augmenter dataset avec transformations

**Composants**:
1. ImageAugmenter - Random transformations
2. ImageMasker - Apply lung masks (optionnel)
3. ImageComparisonVisualizer - Show augmentation effects (NOUVEAU)

In [7]:
print("\n" + "="*80)
print("🎨 PIPELINE 5: DATA AUGMENTATION")
print("="*80)

transformers_list = [
    ('augmenter', ImageAugmenter(
        flip_horizontal=False,  # Medical images: no horizontal flip
        flip_vertical=False,
        rotation_range=15,
        brightness_range=(0.8, 1.2),
        noise_std=0.02,
        zoom_range=(0.95, 1.05),
        probability=0.5,
        seed=RANDOM_STATE,
        verbose=True,
        use_streamlit=False
    )),
    # ImageMasker optionnel (décommenter si besoin)
    # ('masker', ImageMasker(verbose=True, use_streamlit=False)),
]

# Ajouter visualisateur si disponible
if NEW_TRANSFORMERS_AVAILABLE:
    transformers_list.append(
        ('aug_viz', ImageComparisonVisualizer(n_samples=6, comparison_mode='difference',
                                               verbose=True, use_streamlit=False))
    )

pipeline_augmentation = Pipeline(transformers_list)

# Sauvegarder (section, optionnelle)
pipeline_path = os.path.join(MODELS_DIR, 'Section_5_augmentation.pkl')
joblib.dump(pipeline_augmentation, pipeline_path)
print(f"\n✅ Section sauvegardée: {pipeline_path}")
print("💡 Section optionnelle - appliquer sur train set avant preprocessing")

print("\n📋 Structure:")
for idx, (name, transformer) in enumerate(pipeline_augmentation.steps, 1):
    print(f"  {idx}. {name}: {transformer.__class__.__name__}")

print("\n💡 Usage:")
print("  df_augmented = pipeline_augmentation.fit_transform(df_train)")
print("  # Applique: rotation, brightness, noise, zoom")
print("  # Génère: visualisations avant/après/différence")


🎨 PIPELINE 5: DATA AUGMENTATION

✅ Section sauvegardée: ../models/Section_5_augmentation.pkl
💡 Section optionnelle - appliquer sur train set avant preprocessing

📋 Structure:
  1. augmenter: ImageAugmenter
  2. aug_viz: ImageComparisonVisualizer

💡 Usage:
  df_augmented = pipeline_augmentation.fit_transform(df_train)
  # Applique: rotation, brightness, noise, zoom
  # Génère: visualisations avant/après/différence


## 8. 🧠 PIPELINE 6: Preprocessing pour CNN Custom

**Objectif**: Préparer images pour deep learning (128x128, RGB)

**Composants**:
1. ImageAnalyser - Load images
2. ImageResizer - Resize to 128x128
3. ImageNormalizer - Normalize to [0,1]

Note: Garde RGB (3 channels) pour CNN

In [8]:
print("\n" + "="*80)
print("🧠 PIPELINE 6: PREPROCESSING CNN CUSTOM")
print("="*80)

pipeline_preprocessing_cnn = Pipeline([
    ('loader', ImageAnalyser(load_images=True, analyze_masks=False, verbose=True, use_streamlit=False)),
    ('resizer', ImageResizer(img_size=IMG_SIZE_CNN, verbose=True, use_streamlit=False)),
    ('normalizer', ImageNormalizer(verbose=True, use_streamlit=False)),
    # Pas de RGB_to_L - on garde les 3 channels
])

# Sauvegarder (section, pas utilisable seule)
pipeline_path = os.path.join(MODELS_DIR, 'Section_6_preprocessing_cnn.pkl')
joblib.dump(pipeline_preprocessing_cnn, pipeline_path)
print(f"\n✅ Section sauvegardée: {pipeline_path}")
print("⚠️  Section non utilisable seule - utiliser Pipeline_CNN")

print("\n📋 Structure:")
for idx, (name, transformer) in enumerate(pipeline_preprocessing_cnn.steps, 1):
    print(f"  {idx}. {name}: {transformer.__class__.__name__}")

print("\n💡 Usage:")
print("  df_cnn = pipeline_preprocessing_cnn.fit_transform(df_train)")
print("  # Images: 128x128x3, RGB, normalisées")
print("  # Prêt pour CNN custom à 5 blocs")


🧠 PIPELINE 6: PREPROCESSING CNN CUSTOM

✅ Section sauvegardée: ../models/Section_6_preprocessing_cnn.pkl
⚠️  Section non utilisable seule - utiliser Pipeline_CNN

📋 Structure:
  1. loader: ImageAnalyser
  2. resizer: ImageResizer
  3. normalizer: ImageNormalizer

💡 Usage:
  df_cnn = pipeline_preprocessing_cnn.fit_transform(df_train)
  # Images: 128x128x3, RGB, normalisées
  # Prêt pour CNN custom à 5 blocs


## 9. 🔄 PIPELINE 7: Preprocessing pour Transfer Learning

**Objectif**: Préparer images pour VGG16/ResNet/Inception/EfficientNet (224x224)

**Composants**:
1. ImageAnalyser - Load images
2. ImageResizer - Resize to 224x224
3. ImageNormalizer - Normalize to [0,1]

Note: Preprocessing spécifique au modèle (preprocess_input) sera fait plus tard

In [9]:
print("\n" + "="*80)
print("🔄 PIPELINE 7: PREPROCESSING TRANSFER LEARNING")
print("="*80)

pipeline_preprocessing_transfer = Pipeline([
    ('loader', ImageAnalyser(load_images=True, analyze_masks=False, verbose=True, use_streamlit=False)),
    ('resizer', ImageResizer(img_size=IMG_SIZE_TRANSFER, verbose=True, use_streamlit=False)),
    ('normalizer', ImageNormalizer(verbose=True, use_streamlit=False)),
    # Pas de RGB_to_L - modèles pré-entraînés attendent RGB
])

# Sauvegarder (section, pas utilisable seule)
pipeline_path = os.path.join(MODELS_DIR, 'Section_7_preprocessing_transfer.pkl')
joblib.dump(pipeline_preprocessing_transfer, pipeline_path)
print(f"\n✅ Section sauvegardée: {pipeline_path}")
print("⚠️  Section non utilisable seule - utiliser Pipeline_Transfer")

print("\n📋 Structure:")
for idx, (name, transformer) in enumerate(pipeline_preprocessing_transfer.steps, 1):
    print(f"  {idx}. {name}: {transformer.__class__.__name__}")

print("\n💡 Usage:")
print("  df_transfer = pipeline_preprocessing_transfer.fit_transform(df_train)")
print("  # Images: 224x224x3, RGB, normalisées")
print("  # Puis appliquer: keras.applications.<model>.preprocess_input()")


🔄 PIPELINE 7: PREPROCESSING TRANSFER LEARNING

✅ Section sauvegardée: ../models/Section_7_preprocessing_transfer.pkl
⚠️  Section non utilisable seule - utiliser Pipeline_Transfer

📋 Structure:
  1. loader: ImageAnalyser
  2. resizer: ImageResizer
  3. normalizer: ImageNormalizer

💡 Usage:
  df_transfer = pipeline_preprocessing_transfer.fit_transform(df_train)
  # Images: 224x224x3, RGB, normalisées
  # Puis appliquer: keras.applications.<model>.preprocess_input()


## 10. 🎯 PIPELINE 8: Split Train/Validation/Test

**Objectif**: Séparer dataset de manière stratifiée

**Composant**:
1. TrainTestSplitter - Stratified split

Note: Pour deep learning, on fera un second split train → train/val

In [10]:
print("\n" + "="*80)
print("🎯 PIPELINE 8: TRAIN/TEST SPLIT")
print("="*80)

pipeline_split = Pipeline([
    ('splitter', TrainTestSplitter(test_size=TEST_SIZE, random_state=RANDOM_STATE, 
                                   verbose=True, use_streamlit=False)),
])

# Sauvegarder (section utilisable)
pipeline_path = os.path.join(MODELS_DIR, 'Section_8_split.pkl')
joblib.dump(pipeline_split, pipeline_path)
print(f"\n✅ Section sauvegardée: {pipeline_path}")
print("✅ Section utilisable directement sur DataFrame")

print("\n📋 Structure:")
for idx, (name, transformer) in enumerate(pipeline_split.steps, 1):
    print(f"  {idx}. {name}: {transformer.__class__.__name__}")

print("\n💡 Usage:")
print("  splits = pipeline_split.fit_transform(df_metadata)")
print("  df_train, y_train = splits['train']")
print("  df_test, y_test = splits['test']")


🎯 PIPELINE 8: TRAIN/TEST SPLIT

✅ Section sauvegardée: ../models/Section_8_split.pkl
✅ Section utilisable directement sur DataFrame

📋 Structure:
  1. splitter: TrainTestSplitter

💡 Usage:
  splits = pipeline_split.fit_transform(df_metadata)
  df_train, y_train = splits['train']
  df_test, y_test = splits['test']


## 11. 📦 NESTED PIPELINES - Combinaisons Complètes

**Objectif**: Créer pipelines end-to-end pour Streamlit

Ces pipelines combinent plusieurs étapes pour créer des workflows complets.

In [ ]:
print("\n" + "="*80)
print("📦 CRÉATION NESTED PIPELINES")
print("="*80)

# -------------------------
# NESTED 1: ML Complete (Exploration → Preprocessing → PCA)
# -------------------------
print("\n🔗 Nested Pipeline 1: ML Complete (PCA)")

nested_ml_pca = Pipeline([
    ('exploration', pipeline_exploration),
    ('preprocessing', pipeline_preprocessing_ml),
    ('features', pipeline_features_pca),
])

path = os.path.join(MODELS_DIR, 'Pipeline_ML_PCA_Complete.pkl')
joblib.dump(nested_ml_pca, path)
print(f"✅ Pipeline complet sauvegardé: {path}")
print("   📦 UTILISABLE DIRECTEMENT")
print(f"   Étapes totales: {len(pipeline_exploration.steps)} + {len(pipeline_preprocessing_ml.steps)} + {len(pipeline_features_pca.steps)}")

# -------------------------
# NESTED 2: ML Complete (Exploration → Preprocessing → Histogrammes)
# -------------------------
print("\n🔗 Nested Pipeline 2: ML Complete (Histogrammes)")

nested_ml_hist = Pipeline([
    ('exploration', pipeline_exploration),
    ('preprocessing', pipeline_preprocessing_ml),
    ('features', pipeline_features_histogram),
])
path = os.path.join(MODELS_DIR, 'Pipeline_ML_Histogram_Complete.pkl')
path = os.path.join(MODELS_DIR, 'nested_pipeline_ml_hist_complete.pkl')
print(f"✅ Pipeline complet sauvegardé: {path}")
print("   📦 UTILISABLE DIRECTEMENT")
print(f"   Étapes totales: {len(pipeline_exploration.steps)} + {len(pipeline_preprocessing_ml.steps)} + {len(pipeline_features_histogram.steps)}")

# -------------------------
# NESTED 3: CNN Complete (Exploration → Preprocessing CNN)
# -------------------------
print("\n🔗 Nested Pipeline 3: CNN Complete")

nested_cnn = Pipeline([
    ('exploration', pipeline_exploration),
    ('preprocessing', pipeline_preprocessing_cnn),
])
path = os.path.join(MODELS_DIR, 'Pipeline_CNN_Complete.pkl')
path = os.path.join(MODELS_DIR, 'nested_pipeline_cnn_complete.pkl')
print(f"✅ Pipeline complet sauvegardé: {path}")
print("   📦 UTILISABLE DIRECTEMENT")
print(f"   Étapes totales: {len(pipeline_exploration.steps)} + {len(pipeline_preprocessing_cnn.steps)}")

# -------------------------
# NESTED 4: Transfer Learning Complete
# -------------------------
print("\n🔗 Nested Pipeline 4: Transfer Learning Complete")

nested_transfer = Pipeline([
    ('exploration', pipeline_exploration),
    ('preprocessing', pipeline_preprocessing_transfer),
])
path = os.path.join(MODELS_DIR, 'Pipeline_Transfer_Complete.pkl')
path = os.path.join(MODELS_DIR, 'nested_pipeline_transfer_complete.pkl')
print(f"✅ Pipeline complet sauvegardé: {path}")
print("   📦 UTILISABLE DIRECTEMENT")
print(f"   Étapes totales: {len(pipeline_exploration.steps)} + {len(pipeline_preprocessing_transfer.steps)}")

print("\n" + "="*80)
print("✅ TOUS LES NESTED PIPELINES CRÉÉS")

print("="*80)
print("="*80)
print("="*80)

SyntaxError: invalid syntax (2591955270.py, line 73)

## 12. 📊 Résumé des Pipelines

In [ ]:
print("\n" + "="*80)
print("📊 RÉSUMÉ DES PIPELINES V4")
print("="*80)

print("\n" + "="*80)
print("✅ SECTIONS SAUVEGARDÉES (composants)")
print("="*80)

sections_summary = {
    "Section 1 - Exploration": {
        "file": "Section_1_exploration.pkl",
        "usage": "Composant: Analyse dataset",
        "steps": len(pipeline_exploration.steps)
    },
    "Section 2 - Preprocessing ML": {
        "file": "Section_2_preprocessing_ml.pkl",
        "usage": "Composant: Preprocessing ML (64x64, gray)",
        "steps": len(pipeline_preprocessing_ml.steps)
    },
    "Section 3 - Features PCA": {
        "file": "Section_3_features_pca.pkl",
        "usage": "Composant: Extraction PCA",
        "steps": len(pipeline_features_pca.steps)
    },
    "Section 4 - Features Histogrammes": {
        "file": "Section_4_features_histogram.pkl",
        "usage": "Composant: Extraction histogrammes",
        "steps": len(pipeline_features_histogram.steps)
    },
    "Section 5 - Augmentation": {
        "file": "Section_5_augmentation.pkl",
        "usage": "Composant optionnel: Augmentation",
        "steps": len(pipeline_augmentation.steps)
    },
    "Section 6 - Preprocessing CNN": {
        "file": "Section_6_preprocessing_cnn.pkl",
        "usage": "Composant: Preprocessing CNN (128x128, RGB)",
        "steps": len(pipeline_preprocessing_cnn.steps)
    },
    "Section 7 - Preprocessing Transfer": {
        "file": "Section_7_preprocessing_transfer.pkl",
        "usage": "Composant: Preprocessing Transfer (224x224, RGB)",
        "steps": len(pipeline_preprocessing_transfer.steps)
    },
    "Section 8 - Split": {
        "file": "Section_8_split.pkl",
        "usage": "✅ Utilisable: Split train/test",
        "steps": len(pipeline_split.steps)
    },
}

for name, info in sections_summary.items():
    marker = "✅" if "Utilisable" in info['usage'] else "⚙️"
    print(f"\n{marker} {name}")
    print(f"   Fichier: {info['file']}")
    print(f"   Étapes: {info['steps']}")
    print(f"   Usage: {info['usage']}")

print("\n" + "="*80)
print("🎯 PIPELINES COMPLETS (PRÊTS À L'EMPLOI)")
print("="*80)

pipelines_complete = {
    "Pipeline_ML_PCA_Complete": {
        "file": "Pipeline_ML_PCA_Complete.pkl",
        "usage": "ML avec features PCA (RandomForest/XGBoost)",
        "output": "Features PCA (n_samples, 50)",
        "workflow": "Exploration → Preprocessing ML → PCA"
    },
    "Pipeline_ML_Histogram_Complete": {
        "file": "Pipeline_ML_Histogram_Complete.pkl",
        "usage": "ML avec features Histogrammes",
        "output": "Features Histogrammes (n_samples, 32)",
        "workflow": "Exploration → Preprocessing ML → Histogram"
    },
    "Pipeline_CNN_Complete": {
        "file": "Pipeline_CNN_Complete.pkl",
        "usage": "Deep Learning CNN custom",
        "output": "Images 128x128x3 RGB normalisées",
        "workflow": "Exploration → Preprocessing CNN"
    },
    "Pipeline_Transfer_Complete": {
        "file": "Pipeline_Transfer_Complete.pkl",
        "usage": "Transfer Learning (VGG16/ResNet/EfficientNet)",
        "output": "Images 224x224x3 RGB normalisées",
        "workflow": "Exploration → Preprocessing Transfer"
    },
}

for name, info in pipelines_complete.items():
    print(f"\n🚀 {name}")
    print(f"   📦 Fichier: {info['file']}")
    print(f"   🎯 Usage: {info['usage']}")
    print(f"   📤 Output: {info['output']}")
    print(f"   🔄 Workflow: {info['workflow']}")

print("\n" + "="*80)
print("✅ NOTEBOOK V4 TERMINÉ")
print("="*80)
print(f"\n📁 Tous les fichiers sauvegardés dans: {MODELS_DIR}")
print(f"   • Section_*.pkl = Composants (non utilisables seuls)")
print(f"   • Pipeline_*.pkl = Pipelines complets (PRÊTS À L'EMPLOI)")
print(f"\n📁 Outputs dans: {OUTPUTS_DIR}")

print("\n💡 UTILISATION:")
print("  # Charger un pipeline complet")
print("  import joblib")
print("  pipeline = joblib.load('../models/Pipeline_ML_PCA_Complete.pkl')")
print("  ")
print("  # Exécuter end-to-end")
print("  X_features = pipeline.fit_transform(None)")
print("  ")
print("  # Entraîner modèle ML")
print("  from sklearn.ensemble import RandomForestClassifier")
print("  rf = RandomForestClassifier()")
print("  rf.fit(X_features, y)")

timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
print(f"\n⏰ Généré le: {timestamp}")